# 1. ESG 기준원- 기업별 ESG 등급 가져오기 => 평가년도: 2022
- [ESG기준원](http://www.cgs.or.kr/business/esg_tab04.jsp?pg={}&pp=10&skey=&svalue=&sfyear=2021&styear=2021&sgtype=&sgrade=#ui_contents)
- 참고: [DACON : 이제 ESG에 투자하세요! : 기업 ESG 등급을 이용한 국내 ETF ESG 등급 현황](https://dacon.io/competitions/official/235914/codeshare/5669)

## 1.1 필요한 라이브러리 가져오기

In [71]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import time
import urllib.request
# from urllib.request import urlopen   

- 1페이지
http://www.cgs.or.kr/business/esg_tab04.jsp?pg=1&pp=10&skey=&svalue=&sfyear=2021&styear=2021&sgtype=&sgrade=#ui_contents

- 2페이지 
http://www.cgs.or.kr/business/esg_tab04.jsp?pg=2&pp=10&skey=&svalue=&sfyear=2021&styear=2021&sgtype=&sgrade=#ui_contents

In [72]:
#각 테마 페이지별 테마명&주식종목명 가져오기
def page_list(page):
        ESGLIST_URL = "http://www.cgs.or.kr/business/esg_tab04.jsp?pg={}&pp=10&skey=&svalue=&sfyear=2021&styear=2021&sgtype=&sgrade=#ui_contents".format(page) #주소설정
        response = urllib.request.urlopen(ESGLIST_URL)
        ESGLIST_HTML = response.read()

        soup = bs(ESGLIST_HTML)
        STK_ESG_GRADE=pd.DataFrame()
        # 100 페이지까지는 한 페이지에 총 10개 기업 존재
        if page <= 100:
            num_list = [0,1,2,3,4,5,6,7,8,9] 
        # 101 페이지에는 총 5개 기업이 존재
        else:
            num_list = [0,1,2,3,4]
        for i in num_list:
            find_stk = soup.find_all('td')[i*9:i*9+9]
            stk_col_list=[]
            for i in [0,1,2,3,4,5,6,7,8]:
                stk_col_list.append(find_stk[i].text)
                clean_stk_col_list=pd.DataFrame(stk_col_list).transpose()
            STK_ESG_GRADE = pd.concat([STK_ESG_GRADE,clean_stk_col_list])
        return STK_ESG_GRADE

In [73]:
from tqdm import tqdm 
import time

df_esg = pd.DataFrame()
for i in tqdm(list(range(1,102))):
    df_temp = page_list(i)
    df_esg = pd.concat([df_esg,df_temp])
    df_esg

df_esg.columns = ['NO', '기업명','기업코드','ESG등급','환경','사회','지배구조','평가년도','비고']
df_esg.reset_index(inplace=True)
df_esg.drop(columns=['index','NO','비고'], inplace=True)
df_esg.head()

100%|████████████████████████████████████████████████████████████████████████████████| 101/101 [00:08<00:00, 12.57it/s]


,기업명,기업코드,ESG등급,환경,사회,지배구조,평가년도
0,AJ네트웍스,095570,B+,C,B+,B+,2022
1,AK홀딩스,006840,B+,B,A,B,2022
2,AP시스템,265520,D,D,C,C,2022
3,BGF,027410,B+,A,A+,B,2022
4,BGF리테일,282330,A,A,A+,A,2022


In [74]:
df_esg.shape # (1005, 7)

(1005, 7)

In [75]:
df_esg.isnull().sum()

기업명      0
기업코드     0
ESG등급    0
환경       0
사회       0
지배구조     0
평가년도     0
dtype: int64

In [76]:
df_esg["기업명"].nunique()

1005

In [77]:
df_esg.columns

Index(['기업명', '기업코드', 'ESG등급', '환경', '사회', '지배구조', '평가년도'], dtype='object')

In [78]:
df_esg[df_esg["기업명"]=="BNK캐피탈"]

,기업명,기업코드,ESG등급,환경,사회,지배구조,평가년도
6,BNK캐피탈,801190,-,-,-,B+,2022


In [79]:
df_esg[df_esg["기업명"]=="CJ씨푸드"]

,기업명,기업코드,ESG등급,환경,사회,지배구조,평가년도
12,CJ씨푸드,011150,B,C,B,B,2022


In [80]:
df_esg[df_esg["기업명"]=="AJ네트웍스"]

,기업명,기업코드,ESG등급,환경,사회,지배구조,평가년도
0,AJ네트웍스,095570,B+,C,B+,B+,2022


## 1.2 CSV 파일로 저장

In [81]:
df_esg.to_csv("../../data/ESG_Rating(2021).csv", index=False)

In [82]:
pd.read_csv("../../data/ESG_Rating(2021).csv").head(2)

,기업명,기업코드,ESG등급,환경,사회,지배구조,평가년도
0,AJ네트웍스,95570,B+,C,B+,B+,2022
1,AK홀딩스,6840,B+,B,A,B,2022


## 1.3 기업코드 6자리로 반환하기

In [83]:
df_esg['기업코드'] = df_esg['기업코드'].astype('int')

In [84]:
df_esg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   기업명     1005 non-null   object
 1   기업코드    1005 non-null   int32 
 2   ESG등급   1005 non-null   object
 3   환경      1005 non-null   object
 4   사회      1005 non-null   object
 5   지배구조    1005 non-null   object
 6   평가년도    1005 non-null   object
dtypes: int32(1), object(6)
memory usage: 51.2+ KB


In [85]:
def six_digit(x):
    if pd.isna(x) == True:
        pass
    else:
        return '%06d' % x

In [86]:
df_esg['기업코드'] = df_esg['기업코드'].apply(six_digit)
df_esg.sample(5)

,기업명,기업코드,ESG등급,환경,사회,지배구조,평가년도
607,영풍,000670,B,C,B+,B,2022
966,핸즈코퍼레이션,143210,C,C,B,D,2022
497,수산중공업,017550,D,D,D,C,2022
206,노루홀딩스,000320,A,B+,A+,B+,2022
482,세이브존I&C,067830,C,D,C,C,2022


# 2. KRX 정보데이터시스템 - 기업별 ESG 등급 가져오기 => 평가년도: 2011~2018
- [KRX 정보데이터시스템](https://data.krx.co.kr/contents/MDC/HARD/hardController/MDCHARD050.cmd#none)

In [87]:
import requests
import pandas as pd


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
}

df_list = []
for year in range(2011, 2018 + 1):
    data = {'bld': 'dbms/MDC/HARD/MDCHARD05001',
    'locale': 'ko_KR',
    'selTp': '1','grdYy': year}
    response = requests.post('https://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd', headers = headers, data = data)
    df_list.append(pd.DataFrame(response.json()['block1']))
df = pd.concat(df_list).drop(columns = 'RN').reset_index(drop = True)
df

,ISU_NM,GRD1,GRD2,GRD3,GRD4,YY
0,AK홀딩스,B이하,B,B이하,B이하,2011
1,BYC,B이하,B,B이하,B이하,2011
2,CJ,B이하,B+,B이하,,2011
3,CJ CGV,B이하,B,B이하,,2011
4,CJ대한통운,B+,B,B+,B이하,2011
...,...,...,...,...,...,...
5649,샘표,B이하,B,B이하,B이하,2018
5650,샘표식품,B+,B,B+,B+,2018
5651,서연,B이하,B,B이하,B이하,2018
5652,서연이화,B+,B+,B+,B이하,2018


In [88]:
df.columns = ['기업명','ESG등급','환경','사회','지배구조','평가년도']

In [89]:
df

,기업명,ESG등급,환경,사회,지배구조,평가년도
0,AK홀딩스,B이하,B,B이하,B이하,2011
1,BYC,B이하,B,B이하,B이하,2011
2,CJ,B이하,B+,B이하,,2011
3,CJ CGV,B이하,B,B이하,,2011
4,CJ대한통운,B+,B,B+,B이하,2011
...,...,...,...,...,...,...
5649,샘표,B이하,B,B이하,B이하,2018
5650,샘표식품,B+,B,B+,B+,2018
5651,서연,B이하,B,B이하,B이하,2018
5652,서연이화,B+,B+,B+,B이하,2018


In [90]:
df.isnull().sum()

기업명      0
ESG등급    0
환경       0
사회       0
지배구조     0
평가년도     0
dtype: int64

## 2.1 평가년도를 2018년부터 나오도록 정렬

In [91]:
df = df.sort_values(by="평가년도", ascending=False) 

df = df.reset_index(drop=True)
df

,기업명,ESG등급,환경,사회,지배구조,평가년도
0,서울도시가스,B이하,B,B이하,B이하,2018
1,티비에이치글로벌,B이하,B,B이하,B이하,2018
2,평화산업,B이하,B,B이하,B+,2018
3,페이퍼코리아,B이하,C,B이하,B이하,2018
4,퍼시스,B이하,B,B이하,B이하,2018
...,...,...,...,...,...,...
5649,성신양회,B이하,B+,B이하,B이하,2011
5650,성보화학,,B,,,2011
5651,성문전자,B이하,C,B이하,B이하,2011
5652,선창산업,B이하,B,B이하,B이하,2011


## 2.2 csv 파일로 저장

In [92]:
df.to_csv("../../data/ESG_Rating(2011-2018).csv", index=False)

In [93]:
pd.read_csv("../../data/ESG_Rating(2011-2018).csv")

,기업명,ESG등급,환경,사회,지배구조,평가년도
0,서울도시가스,B이하,B,B이하,B이하,2018
1,티비에이치글로벌,B이하,B,B이하,B이하,2018
2,평화산업,B이하,B,B이하,B+,2018
3,페이퍼코리아,B이하,C,B이하,B이하,2018
4,퍼시스,B이하,B,B이하,B이하,2018
...,...,...,...,...,...,...
5649,성신양회,B이하,B+,B이하,B이하,2011
5650,성보화학,NaN,B,NaN,NaN,2011
5651,성문전자,B이하,C,B이하,B이하,2011
5652,선창산업,B이하,B,B이하,B이하,2011


## 2.3 ESG 기준원 데이터와 merge 하기 위해 기업코드 데이터 가져오기
- FinanceDataReader 로 KRX 전체 종목 가져오기

| **국내 주식** |               |   | **미국 주식** |                      |
|:-------------:|---------------|---|:-------------:|----------------------|
|   **Market**  |               |   |   **Market**  |                      |
|      KRX      | KRX 전체 종목 |   |     NASDAQ    |      NASDAQ 종목     |
|     KOSPI     |   KOSPI 종목  |   |      NYSE     | 뉴욕 증권거래소 종목 |
|     KOSDAQ    |  KOSDAQ 종목  |   |      AMEX     |       AMEX 종목      |
|     KONEX     |   KONEX 종목  |   |     SP500     |      S&P500 종목     |

In [94]:
# FinanceDataReader 를 fdr 별칭으로 불러오기
import FinanceDataReader as fdr

fdr.__version__

'0.9.42'

In [95]:
df_krx = fdr.StockListing("KRX")
df_krx.head()

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,060310,KOSDAQ,3S,전자부품 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,김세완,http://www.3sref.com,서울특별시
1,105190,KOSPI,ACE 200,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2,332500,KOSPI,ACE 200TR,NaN,NaN,NaT,NaN,NaN,NaN,NaN
3,448870,KOSPI,ACE 23-12 회사채(AA-이상)액티브,NaN,NaN,NaT,NaN,NaN,NaN,NaN
4,448880,KOSPI,ACE 24-12 회사채(AA-이상)액티브,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [96]:
df_krx = df_krx[['Name','Symbol']]

df_krx.columns = ['Name','기업코드']
df_krx

,Name,기업코드
0,3S,060310
1,ACE 200,105190
2,ACE 200TR,332500
3,ACE 23-12 회사채(AA-이상)액티브,448870
4,ACE 24-12 회사채(AA-이상)액티브,448880
...,...,...
6890,히어로즈 TDF2050액티브,435550
6891,히어로즈 글로벌리츠이지스액티브,437550
6892,히어로즈 단기채권ESG액티브,419890
6893,히어로즈 리츠이지스액티브,429870


## 2.4 KRX 정보데이터시스템 데이터에 기업코드 넣기

In [97]:
df_1118 = df.merge(df_krx, left_on="기업명", right_on="Name", how="left")
df_1118 = df_1118[["기업명", "기업코드", "ESG등급", "환경", "사회", "지배구조", "평가년도"]]
df_1118.head(2)

,기업명,기업코드,ESG등급,환경,사회,지배구조,평가년도
0,서울도시가스,NaN,B이하,B,B이하,B이하,2018
1,티비에이치글로벌,NaN,B이하,B,B이하,B이하,2018


In [98]:
df_1118.shape

(5654, 7)

## 2.5 기업코드에 NaN 값이 있으나 그대로 두기로 함.
- 기업코드가 NaN값인 기업명 일부를 네이버 증권에서 찾아본 결과, 상장되지 않은 기업이었고
- 나중에 KOSPI200 데이터와 합칠 때 NaN 값이 자연스럽게 삭제될 것이라 생각하여
> 기업코드의 NaN 값을 그대로 두기로 함!

In [99]:
df_1118[df_1118['기업코드'].isnull()]

,기업명,기업코드,ESG등급,환경,사회,지배구조,평가년도
0,서울도시가스,NaN,B이하,B,B이하,B이하,2018
1,티비에이치글로벌,NaN,B이하,B,B이하,B이하,2018
17,태양금속공업,NaN,B이하,C,B이하,B+,2018
20,태경화학,NaN,B이하,C,B이하,B이하,2018
23,포스코,NaN,A,A+,A,A,2018
...,...,...,...,...,...,...,...
5630,삼호,NaN,B이하,B,B이하,B이하,2011
5639,세원셀론텍,NaN,B이하,B,B이하,,2011
5643,세아베스틸,NaN,B이하,B,B이하,B이하,2011
5645,선진지주,NaN,B이하,C,B이하,,2011


In [100]:
# df_1118_dn = df_1118.dropna()
# 또는 
# df_1118_dn = df_1118.dropna(axis=0)

In [101]:
# df_1118_dn

In [102]:
# df_1118_dn.info()

# 3. ESG 기준원(df_esg)과 KRX 정보통신데이터(df_1118) 합치기

In [103]:
esg_total = pd.concat([df_esg, df_1118])
esg_total = esg_total.reset_index(drop=True)
esg_total

,기업명,기업코드,ESG등급,환경,사회,지배구조,평가년도
0,AJ네트웍스,095570,B+,C,B+,B+,2022
1,AK홀딩스,006840,B+,B,A,B,2022
2,AP시스템,265520,D,D,C,C,2022
3,BGF,027410,B+,A,A+,B,2022
4,BGF리테일,282330,A,A,A+,A,2022
...,...,...,...,...,...,...,...
6654,성신양회,004980,B이하,B+,B이하,B이하,2011
6655,성보화학,003080,,B,,,2011
6656,성문전자,014910,B이하,C,B이하,B이하,2011
6657,선창산업,NaN,B이하,B,B이하,B이하,2011


## 3.1 완성된 ESG 데이터를 csv 파일로 저장

In [104]:
esg_total.to_csv("../../data/ESG_Rating.csv", index=False)

In [105]:
pd.read_csv("../../data/ESG_Rating.csv")

,기업명,기업코드,ESG등급,환경,사회,지배구조,평가년도
0,AJ네트웍스,95570.0,B+,C,B+,B+,2022
1,AK홀딩스,6840.0,B+,B,A,B,2022
2,AP시스템,265520.0,D,D,C,C,2022
3,BGF,27410.0,B+,A,A+,B,2022
4,BGF리테일,282330.0,A,A,A+,A,2022
...,...,...,...,...,...,...,...
6654,성신양회,4980.0,B이하,B+,B이하,B이하,2011
6655,성보화학,3080.0,NaN,B,NaN,NaN,2011
6656,성문전자,14910.0,B이하,C,B이하,B이하,2011
6657,선창산업,NaN,B이하,B,B이하,B이하,2011
